## Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
Change directory to where this file is located
"""
#%cd 'COPY&PASTE FILE DIRECTORY HERE'

## Import Modules

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

## K-Nearest Neighbor Implementation

Complete (a) the **train**, (b) the **compute_distance** and (c) the **predict_labels** method.
The definitions of various distance metrics used in **compute_distance**, are given as below. Case using the dot product method is already implemented. Complete the remaining cases for using remaining distance metrics.

* Dot product distance: $d(x_i, x_j) = 1 - x_i \cdot x_j$
* Cosine distance: $d(x_i, x_j) = 1 - cos(x_i, x_j)$, where $cos()$ returns the cosine of the angle between two input vectors.
* L1 distance: $d(x_i, x_j) = \sum_{k=1}^{n} |x_{ik} - x_{jk}|$
* L2 (Euclidean) distance: $d(x_i, x_j) = \sqrt{\sum_{k=1}^{n} (x_{ik} - x_{jk})^2}$

In [66]:
class KNN:
    """ k-nearest neighbor classifier class """
    def __init__(self):
        self.X_train = None
        self.y_train = None

    def train(self, X, y):
        """
        Train the classifier using the given training data (X, y).
        Recall that for k-nearest neighbors this is just memorizing the training data.

        Question (a)

        Inputs
        - X: A numpy array of shape (N, D), where N is the number of data points,
            D is the dimensionality of each data point.
        - y: A numpy array of shape (N,) containing the training labels, where
            y[i] is the label for X[i]. With C classes, each y[i] is an integer
            from 0 to C-1.
        """
        ##### YOUR CODE HERE #####
        self.X_train = X
        self.y_train = y
        ##########################

    def inference(self, X_test, k=1, dist_metric='dot'):
        """
        For each test example in X, this method predicts its label by majority vote
        from the k nearest training samples. It returns the predicted labels.

        Do NOT Modify this method.

        Inputs
        - X_test: A numpy array of shape (N, D), where N is the number of test data points,
            D is the dimensionality of each data point.
        - X_train: A numpy array of shape (M, D), where M is the number of training data points,
            D is the dimensionality of each data point.
        - k: The number of neighbors to participate in voting.
            dist_metric: Determines the distance metric to use. The default is dot-product ('dot'),
            but you will need to implement 'L2' for question (b).

        Returns
        - y_pred: A numpy array of shape (N,) containing predicted labels for the test data X,
            where y_pred[i] is the predicted label for the test point X[i].
        """
        dists = self.compute_distance(X_test, dist_metric)
        y_pred = self.predict_labels(X_test, dists, k)
        return y_pred

    def compute_distance(self, X_test, dist_metric='L2'):
        """
        Computes the distance between the training data and test data,
        using dot-product similarity or Euclidean (L2) distance as the distance metric.
        Use self.X_train to retrieve training data.

        Question (b)

        Inputs
        - X_test: A numpy array of shape (N, D), where N is the number of test data points,
            D is the dimensionality of each data point.
        - dist_metric: Determines the distance metric to use.

        Returns
        - dists: A numpy array of shape (N, M) where N is the number of test data points,
            and M is the number of traininig data points, containing distances between
            each pair of test and train data  points based on the given distance metric.
        """

        ##### YOUR CODE HERE #####
        if dist_metric=='Dot':
            dists = np.dot(X_test, self.X_train.T)

        elif dist_metric=='Cos':
            norm_X_test = np.linalg.norm(X_test, axis=1, keepdims=True)
            norm_X_train = np.linalg.norm(self.X_train, axis=1, keepdims=True)
            dot_product = np.dot(X_test, self.X_train.T)
            dists = dot_product / (norm_X_test * norm_X_train)

        elif dist_metric == 'L1':
            dists = np.sum(np.abs(self.X_train - X_test[:, np.newaxis]), axis=2)
            
        elif dist_metric == 'L2':
            dists = np.sqrt(np.sum((self.X_train - X_test[:, np.newaxis])**2, axis=2))

        return dists
        ##########################
    
    def predict_labels(self, X_test, dists, k):
        """
        For the given test image, this method takes a majority vote from k closest points
        to predict the class of the test image.

        Question (c)

        Inputs
        - X_test: A numpy array of shape (N, D), where N is the number of test data points,
            D is the dimensionality of each data point.
        - dists: A numpy array of shape (N, M) where N is the number of test data points,
            and M is the number of traininig data points, containing distances between
            each pair of test and train data points based on the given distance metric.
        - k: The number of neighbors to participate in voting.

        Returns
        - y_pred: A numpy array of shape (N,) containing predicted labels for the test data X,
            where y_pred[i] is the predicted label for the test point X[i].
        """

        ##### YOUR CODE HERE #####
        num_test = X_test.shape[0]
        y_pred = np.zeros(num_test)
        
        for i in range(num_test):
            closest_y = []
            k_closest = np.argsort(dists[i])[:k]
            closest_y = tuple(self.y_train[k_closest])
            vote = Counter(closest_y)
            y_pred[i] = vote.most_common(1)[0][0]
        return y_pred

        ##########################

    def evaluate(self, y, y_hat):
        """
        Compares the predicted labels to the ground truth y, and prints the
        classification accuracy.

        Do NOT Modify this method.

        Inputs
        - y: A numpy array of shape (N,) containing the ground truth labels, where
            N is the number of test examples. With C classes, each y[i] is an integer
            from 0 to C-1.
        - y_hat: A numpy array of shape (N,) containing the predicted labels, where
            N is the number of test examples. With C classes, each y_pred[i] is
            an integer from 0 to C-1.

        Returns:
        - accuracy
        """
        y_hat = np.expand_dims(y_hat, axis=1)
        num_correct = np.sum(y_hat == y)
        accuracy = float(num_correct) / y.shape[0]
        return accuracy

## Data Loading

In [67]:
def sample_data(X, y, count):
    mask = np.random.choice(X.shape[0], count, replace=False)
    X_sampled = X[mask]
    y_sampled = y[mask]
    return X_sampled, y_sampled

In [68]:
num_train_data = 1000
num_test_data = 200

X_train_src, y_train_src, X_test_src, y_test_src = load_data(one_hot_encoding=False) # Training data is flattened when it is loaded
X_train, y_train = sample_data(X_train_src, y_train_src, num_train_data)
X_test, y_test = sample_data(X_test_src, y_test_src, num_test_data)

print()
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 1)
Test data shape: (10000, 784)
Test labels shape: (10000, 1)

Training data shape:  (1000, 784)
Training labels shape:  (1000, 1)
Test data shape:  (200, 784)
Test labels shape:  (200, 1)


## Model Training & Evaluation

In [69]:
model = KNN()
model.train(X_train, y_train)

In [71]:
"""
Model usage for test.
"""
K = 15

y_pred = model.inference(X_test, k=K, dist_metric='Dot')
acc = model.evaluate(y_test, y_pred)
print("Accuarcy:", acc)



TypeError: unhashable type: 'numpy.ndarray'

## Experiments

In [ ]:
# Modify the number of k's and metrics to try as you want
num_ks = 30
metrics = ['Dot', 'Cos', 'L1', 'L2']

In [ ]:
# Run experiments
print_k_interval = 5
result = dict(zip(metrics, [[] for _ in range(len(metrics))]))
for metric in metrics:
    print("running KNN with {} distance metric".format(metric))
    for k in range(1, num_ks+1):
        if k % print_k_interval==0:
            print("    processing... k={:3d}".format(k))
        y_pred = model.inference(X_test, k=k, dist_metric=metric)
        acc = model.evaluate(y_test, y_pred)
        result[metric].append(acc)
    print()

In [ ]:
# Visualize the result
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1,1,1)

x_axis = np.arange(1, num_ks+1, 1)
for i, metric in enumerate(metrics):
    ax.scatter(x_axis, result[metric], label = metric)

ax.set(title="K-Nearest Neighbor Accuracies on different Ks")
ax.set(xlabel='K', ylabel='Accuracy')
ax.set(xticks=np.arange(0, num_ks+1,5), yticks=np.arange(0.5,1.0,0.05))
ax.legend()
plt.show()

**Question (e)**:
- Interpret the result from the above graph. What distance metric would you use for MNIST classification, and why? Can you discuss why that distance is appropriate for MNIST classification?



Your answer: 그래프는 k와 distance metrics의 다양한 조합에 대한 분류 정확도를 보여준다. 일반적으로 k 값에 따라 정확도가 어떻게 변하는지 관찰할 수 있는데, k 값이 작을수록 데이터의 local details를 볼 수 있지만 노이즈에 민감할 수 있다. 반대로 k 값이 클수록 더 정확한 예측을 보여주지만 local patterns를 스무스하게 만들 수 있다.
거리 측정법은 다양한 방식으로 데이터 포인트 간의 유사성을 측정하는데, 그래프는 MNIST 분류에 대해 각 거리 측정 항목이 얼마나 잘 수행되는지 표시해준다.

L2(유클리드) distance는 이미지 분류 작업에 일반적으로 사용된다. 다차원 공간에서 두 점 사이의 직선 거리를 측정한다. 필셀 값의 맥락에서 L2 거리는 로컬과 글로벌 패턴 모두를 고려하여 픽셀 간의 공간적 관계를 표시한다.

Cosine Similarity의 경우 두 벡터 사이의 각도 코사인을 측정한다. 이는 크기 변화에 적합하며 벡터의 크기가 크게 중요하지 않은 작업에 자주 사용된다. 이미지의 경우 절대값보다 픽셀값의 방향이 더 중요할 경우 cosine similarity가 더 효과적일 수 있다.

L2와 cosine 사이의 선택은 데이터의 특성과 모델이 표현하려는 패턴에 따라 달라진다. L2 distance는 이미지 분류 작업에 적합한 기본 방법일 수 있지만 성능은 데이터의 특성에 따라 달라질 수 있다. 또한 특정 데이터 세트에 가장 적합한 조합을 찾기 위해 다양한 distance metrics와 k 값을 가지고 실험하는 것이 일반적이다. 특히 MNIST가 아닌 더 큰 대규모의 데이터 세트를 처리할 떄 각 지표와 관련된 계산 비용을 고려하는 것이 중요하다.

-  If you select the best model based on the results and observe a slight performance drop on unseen datasets, what could be the potential reasons contributing to this outcome? How can you resolve this issue?

Your answer : 데이터 세트에서 모델의 성능이 떨어지면 과적합 때문일 수 있다. 과적합은 모델이 너무 복잡하거나 훈련 데이터의 노이즈를 포착하여 일반화 성능이 저하될 때 발생한다. 과적합을 방지하기 위해 cross validation, regularization, 또는 early stopping과 같은 기술을 사용할 수 있다. 성능 저하의 또 다른 이유는 테스트 세트에 이상한 값이나 노이즈가 있는 데이터가 존재할 수 있기 때문이다. 이러한 경우, 모델을 학습시키기 전에 데이터를 전처리하고 이상값이나 노이즈가 있는 샘플을 제거하는 것이 중요하다.

이 문제를 해결하기 위해 regularization, dropout 또는 early stopping과 같은 기술을 사용하여 과적합을 방지할 수 있다. Regularization은 손실 함수에 페널티 항을 추가하여 큰 weight를 억제하고, 드롭아웃은 훈련 중에 일부 뉴런을 무작위로 탈락시켜 co-adaption을 방지한다. early stopping은 validation error가 개선되지 않을 때 학습 프로세스를 중지하여 모델이 학습 데이터에 과적합되는 것을 방지한다. 또한 모델을 훈련하기 전에 데이터를 전처리하고 이상값이나 노이즈가 있는 샘플을 제거하는 것도 중요하다.